In [2]:
import numpy as np

data_x = np.array([[0.014598, 0.014112], [0.014598, 0.016107131]])

data_sigma = np.array([43584350.6, 43195861.8])  

#data_C = np.array([C1, C2])  


In [3]:
import deepxde as dde


# Define the square domain (e.g., 2x2 square with side length L)
L = 1.0  # Half-length of the square plate
square = dde.geometry.Rectangle([-L, -L], [L, L])

# Define the circular hole at the center (radius R)
R = 0.1  # Radius of the hole
circle = dde.geometry.Disk([0, 0], R)

# Subtract the circular hole from the square to get the final geometry
geom = dde.geometry.CSGDifference(square, circle)

# Create a custom data set in DeepXDE format
data = np.hstack((data_x, data_sigma.reshape(-1, 1)))
# Boundary condition on the outer square (Dirichlet or Neumann)
def boundary_square(x, on_boundary):
    return on_boundary and np.any(np.isclose(x, [-L, -L]) or np.isclose(x, [L, L]))

# Boundary condition on the circular hole (Dirichlet or Neumann)
def boundary_circle(x, on_boundary):
    return on_boundary and np.sqrt(x[0]**2 + x[1]**2) < R + 1e-6 

# Example of a Dirichlet boundary condition (could be Neumann as well)
bc_outer = dde.icbc.DirichletBC(geom, lambda x: 0, boundary_square)
bc_inner = dde.icbc.DirichletBC(geom, lambda x: 1, boundary_circle)  # Example condition

# Define the network for σ
layer_size = [3] + [50] * 3 + [1]  # 3 inputs (x, y, z) -> hidden layers -> 1 output (σ)
activation = "tanh"
initializer = "Glorot uniform"
net_sigma = dde.nn.FNN(layer_size, activation, initializer)

# Define the model
model_sigma = dde.Model(geom, net_sigma)

# Compile the model with data
model_sigma.compile("adam", lr=1e-3)

# Define the custom data
def data_func(x):
    return np.interp(x, data_x, data_sigma)

observations = dde.PointSetBC(data_x, data_sigma, component=0)
data_sigma = dde.data.PointSetBC(data_x, data_sigma)

# Train the model
model_sigma.train(epochs=5000)


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.



Instructions for updating:
non-resource variables are not supported in the long term
Compiling model...
Building feed-forward neural network...


c:\Users\yesda\Documents\GitHub\PINN_testing\.conda\Lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(



'build' took 0.294848 s




AttributeError: 'CSGDifference' object has no attribute 'losses_train'

In [4]:
def pde_C_L(x, C_L):
    C_LA = C_L[:, 0:1]
    C_LB = C_L[:, 1:2]

    x, y, z, t = x[:, 0:1], x[:, 1:2], x[:, 2:3], x[:, 3:4]

    # Use the trained sigma model to predict sigma_A and sigma_B
    sigma_A = model_sigma.predict(x, y, z)  # Predicting sigma_A using trained model_sigma
    sigma_B = model_sigma.predict(x, y, z)  # Assuming similar behavior for B, or train separately

    dC_L_dt = dde.grad.jacobian(C_L, x, i=0, j=3)
    dC_L_dx2 = dde.grad.hessian(C_L, x, i=0, j=0)
    dC_L_dy2 = dde.grad.hessian(C_L, x, i=0, j=1)
    dC_L_dz2 = dde.grad.hessian(C_L, x, i=0, j=2)

    eq1 = C_LA - C_LB * dde.backend.exp(VA/(R*T)*(sigma_A - sigma_B))
    eq2 = dC_L_dt - (1/D_L) * (dC_L_dx2 + dC_L_dy2 + dC_L_dz2)

    return [eq1, eq2]


In [8]:
# Geometry and time domain
geom = dde.geometry.CSGDifference(square, circle)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Network for C_L
layer_size = [4] + [50] * 3 + [2]  # 4 inputs (x, y, z, t) -> hidden layers -> 2 outputs (C_LA, C_LB)
activation = "tanh"
initializer = "Glorot uniform"
net_C_L = dde.nn.FNN(layer_size, activation, initializer)

# Define the initial condition and boundary conditions
#ic = dde.icbc.IC(geomtime)

# Define the model with the PDE, initial and boundary conditions
model_C_L = dde.Model(geomtime, pde_C_L, net_C_L)

# Compile the model with the PDE and dataset
model_C_L.compile("adam", lr=1e-3)

# Train the model
model_C_L.train(epochs=5000)


TypeError: Model.__init__() takes 3 positional arguments but 4 were given

In [ ]:
# Assume C data is in the format: (x, y, z, t) -> C
data_C_L = np.hstack((data_x, t.reshape(-1, 1), data_C.reshape(-1, 1)))  # Combine x, y, z, t with C data

# Adding the C_L data as a point set boundary condition
observations_C_L = dde.PointSetBC(data_x, data_C, component=0)

# Add the C_L data to the model
model_C_L = dde.Model(geomtime, pde_C_L, [ic, observations_C_L], net_C_L)

# Train the model with both PDE and data-driven loss
model_C_L.compile("adam", lr=1e-3)
model_C_L.train(epochs=5000)
